In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from torch.utils.data import random_split,DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import gc
import torchmetrics

In [3]:
class Config:
    IMAGE_SIZE   = (224,224)
    TRAIN_FOLDER = './cat-dog/training_set/'
    TEST_FOLDER  = './cat-dog/test_set/'
    DEVICE       = "cuda:0" if torch.cuda.is_available() else "cpu"
    SPLIT        = (0.7,0.3)
    TRANSFORMS   =  transforms.Compose([transforms.Resize(size=IMAGE_SIZE),
                                       transforms.ToTensor()])
    BATCH_SIZE   = 5
    MODEL_DIR    = 'Pytorch/predefined/'

    torch.hub.set_dir(MODEL_DIR)    

In [13]:
class TrainTestLoader:
    def __init__(self):
        """ DataLoader Class """
        pass
    def get_data(self, data_folder, split_ratio=Config.SPLIT,
                 split=True, batch_size=Config.BATCH_SIZE,
                 transforms=Config.TRANSFORMS):
        if split:
            """ Call method for quick data loading """
            self.train = ImageFolder(data_folder,transform=transforms)
            self.batch_size = batch_size
            self.split_ratio = split_ratio
            tr,ts = int(len(self.train)*split_ratio[0]),len(self.train)-int(len(self.train)*split_ratio[0])
            train_ds,val_ds = random_split(self.train, [tr,ts])
            self.train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
            self.val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=True)
            return self.train_loader,self.val_loader
        else:
            self.test = ImageFolder(data_folder,transform=transforms)
            self.test_loader = DataLoader(self.test)
            return self.test_loader
dl = TrainTestLoader()
train_loader,val_loader = dl.get_data(Config.TRAIN_FOLDER)
test_loader = dl.get_data(Config.TEST_FOLDER,split=False)

# Loading and Modifying Resnet

In [29]:
gc.collect()
resnet    = torch.hub.load('pytorch/vision:v0.10.0','resnet18', pretrained=False)
num_ftrs  = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs,1)
resnet    = resnet.to('cuda')

Using cache found in Pytorch/predefined/pytorch_vision_v0.10.0


# Attaching Model

In [50]:
class ModelFinal(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def make(self, resnet):
        self.resnet = resnet
    def train(self, X_cuda, y_cuda):
        out = self(X_cuda)
        return torch.nn.functional.binary_cross_entropy(out,y_cuda)
    def fit(self, train_loader, epochs=1, lr=0.001):
        opt = torch.optim.Adam(self.parameters(),lr=lr)
        for i in range(epochs):
            iterator_loader = tqdm.tqdm(train_loader, desc='Batch', total=1121)
            for batch in iterator_loader:
                X_cuda = batch[0].to(dtype=torch.float32, device=Config.DEVICE)
                y_cuda = batch[1].to(dtype=torch.float32, device=Config.DEVICE).unsqueeze(1)
                loss = self.train(X_cuda, y_cuda)
                loss.backward()
                opt.step()
                opt.zero_grad()
                torch.cuda.empty_cache()
            print()
            gc.collect()
    # @property
    # def device(self):
    #     return self.get_device()
    def forward(self, X):
        model_output = self.resnet(X)
        softmax_output = torch.nn.Sigmoid()(model_output)
        return softmax_output
    def predict(self,X):
        # self.eval()
        with torch.no_grad():
            return self(X)

In [51]:
del model_attached
gc.collect()

1971

In [52]:
model_attached = ModelFinal()
model_attached.make(resnet=resnet)

In [53]:
model_attached.fit(train_loader=train_loader)

Batch: 100%|████████████████████████████████| 1121/1121 [02:40<00:00,  6.99it/s]

In [54]:
for x,y in val_loader:
    break

In [55]:
x.shape

torch.Size([5, 3, 224, 224])

In [56]:
x.device

device(type='cpu')

In [57]:
y_pred = model_attached.predict(x.to(device=Config.DEVICE))

In [58]:
y_pred.shape

torch.Size([5, 1])

In [59]:
y_pred

tensor([[0.4551],
        [0.4991],
        [0.4903],
        [0.5212],
        [0.5168]], device='cuda:0')